In [70]:
%%capture
%pip install qiskit pylatexenc

In [71]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram

simulator = StatevectorSampler()

In [72]:
q = QuantumRegister(3, name='q')

## Alínea (1)

In [73]:
# Criar circuito do oráculo

## Alínea (2)

In [74]:
# Criar circuito do operador de difusão

## Alínea (3)

In [78]:
# Criar circuito de Grover aqui

In [76]:
# Simulação
job = simulator.run([qc], shots=1024)
result = job.result()[0]
counts = result.data.meas.get_counts()

In [ ]:
# Visualização
plot_histogram(counts)